In [21]:
PPMC_ORDER = 4
ID = 0
TEXT = 9
PREDICTION = 6

import csv
from PPMC import PPMC

text = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."

trainRDD = sc.textFile("train_sin_repeticiones.csv").mapPartitions(lambda x: csv.reader(x))
trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

#se podria hacer un map antes que limpie el texto
#comprimo todo el train con ppmc
trainRDD = trainRDD.map(lambda line: (line[PREDICTION], (PPMC(PPMC_ORDER), line[TEXT])))

trainRDD = trainRDD.map(lambda line: (line[0], (line[1][0], 1, line[1][0].compress(line[1][1]))))
    
#comprimo el texto de prueba con los modelos de todo el rdd
trainRDD = trainRDD.map(lambda line: (line[0], (len(line[1][0].compress(text)), line[1][1])))

print trainRDD.take(2)

[('5', (68, 1)), ('5', (59, 1))]


In [24]:

trainRDD = trainRDD.reduceByKey(lambda value1, value2: (value1[0]+value2[0], value1[1]+value2[1]))

trainRDD = trainRDD.filter(lambda line1, line2: line1 if line1[1][0]/line1[1][1] < line2[1][0]/line2[1][1] else line2)

print trainRDD.take(2)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 4.0 failed 1 times, most recent failure: Lost task 3.0 in stage 4.0 (TID 7, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/usr/lib/spark-1.6.0-bin-hadoop2.6/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/usr/lib/spark-1.6.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark-1.6.0-bin-hadoop2.6/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/usr/lib/spark-1.6.0-bin-hadoop2.6/python/pyspark/rdd.py", lin